In [13]:
# Thêm autoreload vào để tự động reload lại module nếu có thay đổi code trong module
%load_ext autoreload
%autoreload 2

import mediapipe as mp
import numpy as np
import pandas as pd
import copy
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.training_storage_helper import load_model

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# RF_model.pkl là sau điều chỉnh tham số
RF_model = load_model('../best_models/random_forest.pkl')

# Load input scaler
input_scaler = load_model("../best_models/input_scaler.pkl")

In [15]:
IMPORTANT_LMS = [
    "nose",
    "left_shoulder",
    "right_shoulder",
    "left_elbow",
    "right_elbow",
    "left_pinky",
    "right_pinky",
    "left_index",
    "right_index",
    "left_hip",
    "right_hip",
    "left_knee",
    "right_knee",
    "left_foot_index",
    "right_foot_index",
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")

In [16]:
def draw_landmarks(mp_drawing, mp_pose, image: np.array, pose_landmarks):
    """
    Vẽ landmarks lên ảnh kèm theo tọa độ của từng landmark.
    """
    # Vẽ các landmarks và kết nối
    mp_drawing.draw_landmarks(
        image,
        pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(
            color=(255, 0, 0),  # Màu của landmark
            thickness=5,  # Độ dày của landmark
            circle_radius=5,  # Bán kính điểm landmark
        ),
        connection_drawing_spec=mp_drawing.DrawingSpec(
            color=(0, 255, 0),  # Màu của đường kết nối giữa các landmark
            thickness=2,  # Độ dày của đường nối
        ),
    )

    # Vẽ tọa độ của từng landmark lên ảnh
    image_height, image_width, _ = image.shape
    for idx, landmark in enumerate(pose_landmarks.landmark):
        # Chuyển đổi toạ độ tương đối (từ 0-1) thành toạ độ pixel
        x = int(landmark.x * image_width)
        y = int(landmark.y * image_height)
        
        # Vẽ tọa độ lên ảnh
        cv2.putText(
            image,
            f'({landmark.x}, {landmark.x})',  # Văn bản là toạ độ x, y
            (x, y),  # Vị trí đặt văn bản
            cv2.FONT_HERSHEY_SIMPLEX,  # Font chữ
            0.5,  # Kích thước chữ
            (255, 255, 255),  # Màu chữ (trắng)
            1,  # Độ dày của nét chữ
            cv2.LINE_AA
        )

        break

    # Lưu ảnh với landmarks và tọa độ
    cv2.imwrite("output_with_coordinates.jpg", image)


In [17]:
def square_for_image(image: np.array):
    # Nếu hình ảnh là một đối tượng NumPy (cv2 image)
    if isinstance(image, np.ndarray):
        # Chuyển đổi từ định dạng cv2 (NumPy array) sang PIL image để xử lý
        image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        # Nếu hình ảnh đã ở định dạng PIL thì không cần chuyển đổi
        image_pil = image

    # Lấy kích thước ảnh gốc
    original_size = image_pil.size

    # Kích thước mới sẽ là kích thước lớn nhất giữa chiều rộng và chiều cao
    max_width = max(original_size)
    new_size = (max_width, max_width)

    # Tạo một ảnh mới với nền đen (kích thước vuông)
    new_image = Image.new("RGB", new_size, (0, 0, 0))

    # Dán ảnh gốc vào ảnh mới với khoảng trống màu đen
    new_image.paste(image_pil, 
                    ((max_width - original_size[0]) // 2, (max_width - original_size[1]) // 2))

    # Chuyển ảnh mới từ định dạng PIL sang định dạng NumPy (cv2)
    new_image_cv2 = cv2.cvtColor(np.array(new_image), cv2.COLOR_RGB2BGR)

    return new_image_cv2, new_size

def process_image(image):
    """Load and pre-process the image."""
    image, new_size = square_for_image(image)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB), new_size

In [18]:
def calculate_phase_difference(OA: tuple, OB: tuple) -> float:
    """
    Calculate the phase difference (in degrees) of vector OB relative to vector OA.
    :param OA: A tuple representing the first vector (OA).
    :param OB: A tuple representing the second vector (OB).
    :return: Phase difference in degrees from 0 to 360.
    """
    # Calculate the dot product and magnitudes
    dot_product = OA[0] * OB[0] + OA[1] * OB[1]
    norm_OA = np.sqrt(OA[0] ** 2 + OA[1] ** 2)
    norm_OB = np.sqrt(OB[0] ** 2 + OB[1] ** 2)

    # Calculate the cosine of the angle
    cos_theta = dot_product / (norm_OA * norm_OB)
    
    # Ensure the value is within the valid range for arccos due to floating-point errors
    cos_theta = np.clip(cos_theta, -1, 1)
    
    # Calculate the angle in radians
    theta = np.arccos(cos_theta)
    
    # Calculate the cross product (only the z-component for 2D vectors)
    cross_product = OA[0] * OB[1] - OA[1] * OB[0]
    
    # Adjust the angle based on the direction
    if cross_product < 0:
        theta = 2 * np.pi - theta

    # Convert the angle to degrees
    theta_degrees = np.degrees(theta)

    return theta_degrees

In [19]:
def rotate_point(point: tuple, center: tuple, angle: float, origin_size = (612, 408)) -> tuple:
    x, y = point
    cx, cy = center

    x_new = (x - cx) * np.cos(np.radians(angle)) * origin_size[0] - (y - cy) * np.sin(np.radians(angle)) * origin_size[1] + cx * origin_size[0]
    y_new = (x - cx) * np.sin(np.radians(angle)) * origin_size[0] + (y - cy) * np.cos(np.radians(angle)) * origin_size[1] + cy * origin_size[1]

    return x_new / origin_size[0], y_new / origin_size[1]

In [20]:
def rotate_keypoints(keypoints, image_path: str, origin_size = (612, 408)) -> np.array:
    left_shoulder = keypoints.landmark[11]
    right_shoulder = keypoints.landmark[12]
    left_hip = keypoints.landmark[23]
    right_hip = keypoints.landmark[24]

    center_shoulder = (left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2
    center_hip = (left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2

    center = (center_shoulder[0] + center_hip[0]) / 2, (center_shoulder[1] + center_hip[1]) / 2

    O_center_shoulder = (center_shoulder[0] - center[0], center_shoulder[1] - center[1])
    Oy = (0, -1)

    theta = calculate_phase_difference(O_center_shoulder, Oy)

    # rotate each key point
    for point in keypoints.landmark:
        point_rotated = rotate_point((point.x, point.y), center, theta, origin_size)
        point.x = point_rotated[0]
        point.y = point_rotated[1]

    return keypoints

In [21]:
def find_center_of_mass(df_key_points):
    left_hip = (df_key_points["left_hip_x"], df_key_points["left_hip_y"])
    right_hip = (df_key_points["right_hip_x"], df_key_points["right_hip_y"])
    
    # Find the center of the hips
    center_hip = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)

    # Get the nose coordinates
    nose = (df_key_points["nose_x"], df_key_points["nose_y"])

    return (center_hip[0] + nose[0]) / 2, (center_hip[1] + nose[1]) / 2

def calculate_scale_value(df_key_points):
    """Calculate the scale value based on nose and hip points."""
    left_hip = (df_key_points["left_hip_x"], df_key_points["left_hip_y"])
    right_hip = (df_key_points["right_hip_x"], df_key_points["right_hip_y"])
    
    # Find the center of the hips
    center_hip = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)

    # Get the nose coordinates
    nose = (df_key_points["nose_x"], df_key_points["nose_y"])

    # Calculate the distance between nose and center of the hips
    distance = np.sqrt((center_hip[0] - nose[0])**2 + (center_hip[1] - nose[1])**2)

    # Scale value based on distance
    return 0.5 / distance

def scale_keypoints(df_key_points, scale_value):
    """Scale all key points based on the calculated scale value."""
    for landmark in IMPORTANT_LMS:
        df_key_points[f"{landmark}_x"] *= scale_value
        df_key_points[f"{landmark}_y"] *= scale_value
    return df_key_points

In [22]:
def extract_and_recalculate_landmarks(pose_landmarks):
        # Prepare data for CSV
        columns_name = [] + [f"{landmark}_{axis}" for landmark in IMPORTANT_LMS for axis in ['x', 'y', 'z']]
        columns_value = []

        for id, landmark in enumerate(pose_landmarks):
            if mp_pose.PoseLandmark(id).name.lower() in IMPORTANT_LMS:
                # Adjust keypoint coordinates
                columns_value.extend([landmark.x, landmark.y, landmark.z])

        df_key_points = pd.DataFrame([columns_value], columns=columns_name)

        center = find_center_of_mass(df_key_points)
        shifting = (0.5 - center[0], 0.5 - center[1])

        for landmark in IMPORTANT_LMS:
            df_key_points[f"{landmark}_x"] += shifting[0]
            df_key_points[f"{landmark}_y"] += shifting[1]

        # Calculate the scale value based on the distance between nose and hip
        scale_value = calculate_scale_value(df_key_points)

        # Scale keypoints
        df_key_points = scale_keypoints(df_key_points, scale_value)

        return df_key_points

In [23]:
def predict(image, model, input_scaler, prediction_probability_threshold=0.5) -> str:
    current_class = "Unknown"

    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        
        # Chuyển ảnh sang định dạng RGB để xử lý
        image, new_size = process_image(image)
        results = pose.process(image)

        if not results.pose_landmarks:
            return "No human found"

        initial_pose_landmarks = copy.deepcopy(results.pose_landmarks)
        image.flags.writeable = True

        # Khôi phục lại màu gốc của ảnh
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Lấy key points từ landmarks
        try:
            results.pose_landmarks = rotate_keypoints(results.pose_landmarks, image, new_size)

            draw_landmarks(mp_drawing, mp_pose, image, results.pose_landmarks)

            # draw_landmarks(mp_drawing, mp_pose, image, results.pose_landmarks)

            key_points_df = extract_and_recalculate_landmarks(results.pose_landmarks.landmark)

            # Convert DataFrame to numpy array
            key_points = key_points_df.values.reshape(1, -1)  # Chuyển DataFrame thành mảng 2D với đúng kích thước

            # Scale input trước khi dự đoán
            X = input_scaler.transform(key_points)

            print(f"Input shape: {X}")

            # Dự đoán
            predicted_class = model.predict(X)[0]  # Dự đoán dựa trên mô hình và input đã được scale

            print(f"Predicted class: {predicted_class}")

            return str(predicted_class)

        except Exception as e:
            print(f"Error: {e}")
            return "Prediction failed"


In [24]:
image = cv2.imread("../../data/3_procumbent/10.jpg")
predict(image, RF_model, input_scaler)

Input shape: [[-0.51905714 -0.61041598  1.1154815  -1.01178288 -0.7484873   0.5855182
  -0.08755261 -0.68534348 -0.44256557 -0.83357406 -0.79738069  0.64675547
   0.674953   -0.68476105 -0.13140241 -0.73002222 -1.00928667  0.72134969
   0.40917208 -0.96867529  0.55808752 -0.78091566 -0.89867868  0.76905657
   0.31865253 -0.94635789  0.61221037 -0.83682618 -0.59588325  0.91015253
  -0.391318   -0.5706534  -0.9142671  -0.41732858 -0.21112798  0.46421219
   0.28391182 -0.21551948 -0.08794334 -0.1935138  -0.03160482 -0.28087542
  -0.13182053  0.01012701 -0.64083423]]
Predicted class: 2


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


'2'